In [1]:
import pandas as pd
import pulp
import numpy as np
import pickle
import os 

from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

#выбор рабочей папки и данных
папка="""C:\КОНКУРС"""
os.chdir(папка)
таблица="""train20.xlsx"""
лист='data20'

#---- ЗАГРУЗКА ДАННЫХ ----
df=pd.read_excel(таблица,sheet_name=лист)
print(df.head())

X_col=df.columns[0:20]
Y_col=df.columns[20:]

X=df[X_col]
X=X.fillna(0)

Y=df[Y_col]
Y=Y.fillna(0)


test_data=pd.read_excel(таблица,sheet_name='control20')
test_data=test_data.set_index('ID')


C:\Users\Zver\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)


   x1  x8  x9  x11  x13  x14  x16  x20  x21  x22  ...  z1  z2  z3  z4  z5  \
0   0   3   0    0    0    0    0    1    0    1  ...   0   0   0   0   0   
1   0   4   0    0    0    0    0    1    0    0  ...   0   0   0   0   0   
2   0   4   0    1    1    0    0    0    0    0  ...   1   0   0   0   0   
3   0   4   0    1    1    0    0    1    0    0  ...   1   1   0   0   0   
4   0   4   0    1    0    0    0    1    0    0  ...   1   0   0   0   0   

   zz0  zz1  zz2  zz3  zz4  
0    1    0    0    0    0  
1    1    0    0    0    0  
2    0    1    0    0    0  
3    0    0    1    0    0  
4    0    1    0    0    0  

[5 rows x 30 columns]


In [2]:
#обучение и сохранение модели
#classifiers = [LinearDiscriminantAnalysis(), # линейное разделение
#              SVC(kernel='linear'), # метод опорных векторов с линейным ядром
#              LogisticRegression(), # логистическая регрессия
#              KNeighborsClassifier(n_neighbors=6), # метод ближайших соседей
#              DecisionTreeClassifier(),#дерево решений
#              RandomForestClassifier(n_estimators=10),# случайный лес
#              GaussianNB()#наивный байесовский

clf=RandomForestClassifier(n_jobs=-1,max_depth=5,n_estimators=200,random_state=42)


res=[]

plavila={}

for i in Y.columns:
    
    #обучаемся на тест_трейне, для оценки точности методов
    X_train, X_test, y_train, y_test = train_test_split(X, Y[i], test_size=0.20, random_state=42)
     
    clf.fit(X=X_train,y=y_train)
   

    y_train_predict_proba=clf.predict_proba(X_train)
    y_train_predict=clf.predict(X_train)
    y_test_predict_proba=clf.predict_proba(X_test)
    y_test_predict=clf.predict(X_test)    
    
    res+=[[i,
          accuracy_score(y_train,y_train_predict),
          roc_auc_score(y_train,y_train_predict_proba[:,1]),
          accuracy_score(y_test,y_test_predict),
          roc_auc_score(y_test,y_test_predict_proba[:,1]),
         ]]
    
    #обучаемся на всем множестве данных
    clf.fit(X=X,y=Y[i])
    test_data[i]=clf.predict(test_data[X.columns])
    
res=pd.DataFrame(res,columns=['diagnoz','acc_train','roc_auc_train','acc_test','roc_auc_test'])
test_data.to_excel('РАЗМЕТКА_RF_0407.xlsx')
print('РАСЧЕТЫ ЗАКОНЧЕНЫ, РЕЗУЛЬТАТЫ СОХРАНЕНЫ')

РАСЧЕТЫ ЗАКОНЧЕНЫ, РЕЗУЛЬТАТЫ СОХРАНЕНЫ
